<a href="https://colab.research.google.com/github/rakhira/texas-foster-care-adoption-classifier/blob/master/Copy_of_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
import pandas as pd

In [2]:
# create all dataframes

csv_names = ['CPI_3.8_Abuse_Neglect_Investigations_-_Victims_with_Demographics_by_Region_FY2010-FY2019', 
            'CPS_2.1_Removals_-_by_Region_with_Child_Demographics_FY2010-2019', 
            'CPS_3.2_in_Substitute_Care_on_August_31_by_Placement_Type_with_Demographics_FY2010-2019', 
            'CPS_4.1_Adoption_-_Children_Waiting_For_Adoption_on_31_August_by_Region_with_Demographics_FY2010-2019', 
            'CPS_4.3_Adoption_-_Children_In_Adoption_Placements_by_Region_with_Demographics_FY2010-2019', 
            'CPS_4.4_Adoption_-_Disabling_Conditions_of_Children_in_Adoption_Placements_by_Region_with_Demographics_FY2010-2019', 
            'CPS_4.5_Adoptions_Consummated_by_Region_with_Demographics_FY2010-2019']

dfs = {}
for i, csv in enumerate(csv_names):
  dfs[i] = pd.read_csv(f'{csv}.csv')

In [3]:
# COME UP WITH A BETTER WAY TO DO THIS

df0 = dfs[0] #done
df1 = dfs[1] #done
df2 = dfs[2] #done
df3 = dfs[3]
df4 = dfs[4]
df5 = dfs[5]
df6 = dfs[6]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
final_df = df2.copy()

In [7]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144745 entries, 0 to 144744
Data columns (total 10 columns):
 #   Column                                   Non-Null Count   Dtype 
---  ------                                   --------------   ----- 
 0   Fiscal Year                              144745 non-null  int64 
 1   County                                   144745 non-null  object
 2   Region                                   144745 non-null  object
 3   Gender                                   144745 non-null  object
 4   Race_Ethnicity                           144745 non-null  object
 5   Age                                      144745 non-null  int64 
 6   Type of Substitute Care                  144745 non-null  object
 7   Placement Type                           144745 non-null  object
 8   Relative Placement                       144745 non-null  object
 9   Children in Subsitute Care on 31 August  144745 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 11.0+ M

In [8]:
final_df.drop(['County'], axis=1, inplace=True)

In [9]:
# bucketize the 'Age' column into 'Age Group'

bins = [0, 5, 12, 17]
labels = ['Birth to Five Years Old', 'Six to Twelve Years Old', 'Thirteen to Seventeen Years Old']
final_df['Age Group'] = pd.cut(final_df['Age'], bins, labels=labels)
final_df.drop(['Age'], axis=1, inplace=True)

In [10]:
# rename the 'Race_Ethnicity' column

final_df.rename(columns={'Race_Ethnicity': 'Race/Ethnicity'}, index={'ONE': 'one'}, inplace=True)

In [11]:
final_df1 = pd.pivot_table(final_df, values='Relative Placement', index = ['Fiscal Year', 'Region', 'Gender', 'Race/Ethnicity', 'Age Group'], columns=['Type of Substitute Care'], aggfunc='count')

In [12]:
final_df1

Type of Substitute Care                                                        Foster Care  Other Substitute Care
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                                          
2010        1-Lubbock Female African American Birth to Five Years Old                 24.0                   15.0
                                              Six to Twelve Years Old                 24.0                    8.0
                                              Thirteen to Seventeen Years Old         32.0                    3.0
                             Anglo            Birth to Five Years Old                 40.0                   18.0
                                              Six to Twelve Years Old                 46.0                   17.0
...                                                                                    ...                    ...
2019        9-Midland Male   Native American  Birth to Five Years Old                  NaN                    2.0
                                              Six to Twelve Years Old                  1.0                    NaN
                             Other            Birth to Five Years Old                  4.0                    7.0
                                              Six to Twelve Years Old                  7.0                    3.0
                                              Thirteen to Seventeen Years Old          1.0                    1.0

[3090 rows x 2 columns]

In [13]:
final_df2 = pd.pivot_table(final_df, values='Type of Substitute Care', index = ['Fiscal Year', 'Region', 'Gender', 'Race/Ethnicity', 'Age Group'], columns=['Relative Placement'], aggfunc='count')

In [14]:
final_df2.head()

Relative Placement                                                             Non-Relative  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                              
2010        1-Lubbock Female African American Birth to Five Years Old                  25.0      14.0
                                              Six to Twelve Years Old                  26.0       6.0
                                              Thirteen to Seventeen Years Old          34.0       1.0
                             Anglo            Birth to Five Years Old                  40.0      18.0
                                              Six to Twelve Years Old                  45.0      18.0

In [15]:
final_df3 = final_df1.merge(final_df2, on=['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group'])

In [16]:
final_df3.head()

Foster Care  ...  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                     ...          
2010        1-Lubbock Female African American Birth to Five Years Old                 24.0  ...      14.0
                                              Six to Twelve Years Old                 24.0  ...       6.0
                                              Thirteen to Seventeen Years Old         32.0  ...       1.0
                             Anglo            Birth to Five Years Old                 40.0  ...      18.0
                                              Six to Twelve Years Old                 46.0  ...      18.0

[5 rows x 4 columns]

In [17]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45219 entries, 0 to 45218
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Fiscal Year        45219 non-null  int64 
 1   Region Code        45219 non-null  int64 
 2   Region             45219 non-null  object
 3   Confirmed Victims  45219 non-null  object
 4   Gender             45219 non-null  object
 5   Race/Ethnicity     45219 non-null  object
 6   Age                45219 non-null  object
 7   Victims            45219 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


In [18]:
df0.head()

,Fiscal Year,Region Code,Region,Confirmed Victims,Gender,Race/Ethnicity,Age,Victims
0,2019,1,1-Lubbock,Not Confirmed,Female,African American,0,43
1,2019,1,1-Lubbock,Not Confirmed,Female,African American,1,35
2,2019,1,1-Lubbock,Not Confirmed,Female,African American,10,10
3,2019,1,1-Lubbock,Not Confirmed,Female,African American,11,13
4,2019,1,1-Lubbock,Not Confirmed,Female,African American,12,12


In [19]:
df0 = df0[df0['Age'] != 'Over 18']
df0 = df0[df0['Age'] != 'Unknown']
df0['Age'] = df0['Age'].astype(float)

In [20]:
df0.drop(['Region Code'], axis=1, inplace=True)

In [21]:
df0.drop(['Confirmed Victims'], axis=1, inplace=True)

In [22]:
# bucketize the 'Age' column into 'Age Group'

bins = [0, 5, 12, 17]
labels = ['Birth to Five Years Old', 'Six to Twelve Years Old', 'Thirteen to Seventeen Years Old']
df0['Age Group'] = pd.cut(df0['Age'], bins, labels=labels)
df0.drop(['Age'], axis=1, inplace=True)

In [23]:
df0_2 = df0.groupby(['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group']).count()['Victims']

In [24]:
df0_2.reset_index

<bound method Series.reset_index of Fiscal Year  Region        Gender   Race/Ethnicity    Age Group                      
2010         1-Lubbock     Female   African American  Birth to Five Years Old            10.0
                                                      Six to Twelve Years Old            14.0
                                                      Thirteen to Seventeen Years Old    10.0
                                    Anglo             Birth to Five Years Old            10.0
                                                      Six to Twelve Years Old            14.0
                                                                                         ... 
2019         Out of State  Unknown  Native American   Six to Twelve Years Old             NaN
                                                      Thirteen to Seventeen Years Old     NaN
                                    Other             Birth to Five Years Old             NaN
                                

In [25]:
final_df3.reset_index

<bound method DataFrame.reset_index of                                                                                Foster Care  ...  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                     ...          
2010        1-Lubbock Female African American Birth to Five Years Old                 24.0  ...      14.0
                                              Six to Twelve Years Old                 24.0  ...       6.0
                                              Thirteen to Seventeen Years Old         32.0  ...       1.0
                             Anglo            Birth to Five Years Old                 40.0  ...      18.0
                                              Six to Twelve Years Old                 46.0  ...      18.0
...                                                                                    ...  ...       ...
2019        9-Midland Male   Native American  Birth to Five Years Old                  NaN  ...       2.0
       

In [26]:
final_df4 = pd.merge(df0_2, final_df3, on = ['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group'])

In [27]:
final_df4.head()

Victims  ...  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                 ...          
2010        1-Lubbock Female African American Birth to Five Years Old             10.0  ...      14.0
                                              Six to Twelve Years Old             14.0  ...       6.0
                                              Thirteen to Seventeen Years Old     10.0  ...       1.0
                             Anglo            Birth to Five Years Old             10.0  ...      18.0
                                              Six to Twelve Years Old             14.0  ...      18.0

[5 rows x 5 columns]

In [28]:
df1.head()

,Fiscal Year,Region,Removal Stage,Gender,Race/Ethnicity,Age,Removals
0,2019,1-Lubbock,Family Preservation,Female,African American,0,1
1,2019,1-Lubbock,Family Preservation,Female,African American,1,1
2,2019,1-Lubbock,Family Preservation,Female,African American,2,1
3,2019,1-Lubbock,Family Preservation,Female,African American,3,0
4,2019,1-Lubbock,Family Preservation,Female,African American,4,0


In [29]:
# bucketize the 'Age' column into 'Age Group'

bins = [0, 5, 12, 17]
labels = ['Birth to Five Years Old', 'Six to Twelve Years Old', 'Thirteen to Seventeen Years Old']
df1['Age Group'] = pd.cut(df1['Age'], bins, labels=labels)
df1.drop(['Age'], axis=1, inplace=True)

In [30]:
df1.head()

,Fiscal Year,Region,Removal Stage,Gender,Race/Ethnicity,Removals,Age Group
0,2019,1-Lubbock,Family Preservation,Female,African American,1,NaN
1,2019,1-Lubbock,Family Preservation,Female,African American,1,Birth to Five Years Old
2,2019,1-Lubbock,Family Preservation,Female,African American,1,Birth to Five Years Old
3,2019,1-Lubbock,Family Preservation,Female,African American,0,Birth to Five Years Old
4,2019,1-Lubbock,Family Preservation,Female,African American,0,Birth to Five Years Old


In [31]:
df1.drop(['Removal Stage'], axis=1, inplace=True)

In [32]:
df1_2 = df1.groupby(['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group']).count()['Removals']

In [33]:
final_df5 = pd.merge(df1_2, final_df4, on = ['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group'])

In [34]:
final_df5.head()

Removals  ...  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                  ...          
2010        1-Lubbock Female African American Birth to Five Years Old              10.0  ...      14.0
                                              Six to Twelve Years Old              12.0  ...       6.0
                                              Thirteen to Seventeen Years Old       6.0  ...       1.0
                             Anglo            Birth to Five Years Old              10.0  ...      18.0
                                              Six to Twelve Years Old              14.0  ...      18.0

[5 rows x 6 columns]

In [35]:
df3.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Chidlren Waiting on Adoption 31 August,Average Months since Termination of Parental Rights
0,2019,1-Lubbock,Female,African American,Birth to 5 Years Old,1,4
1,2019,1-Lubbock,Female,African American,Birth to 5 Years Old,5,5
2,2019,1-Lubbock,Female,African American,6-12 Years Old,4,48
3,2019,1-Lubbock,Female,African American,13-17 Years Old,3,59
4,2019,1-Lubbock,Female,African American,13-17 Years Old,3,22


In [36]:
df3['Total Months Since Termination'] = df3['Chidlren Waiting on Adoption 31 August'] * df3['Average Months since Termination of Parental Rights']

In [37]:
df3.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Chidlren Waiting on Adoption 31 August,Average Months since Termination of Parental Rights,Total Months Since Termination
0,2019,1-Lubbock,Female,African American,Birth to 5 Years Old,1,4,4
1,2019,1-Lubbock,Female,African American,Birth to 5 Years Old,5,5,25
2,2019,1-Lubbock,Female,African American,6-12 Years Old,4,48,192
3,2019,1-Lubbock,Female,African American,13-17 Years Old,3,59,177
4,2019,1-Lubbock,Female,African American,13-17 Years Old,3,22,66


In [44]:
# need to change age group labels to match final_dfs

df3['Age Group'].replace({'Birth to 5 Years Old': 'Birth to Five Years Old', '6-12 Years Old': 'Six to Twelve Years Old', '13-17 Years Old':  'Thirteen to Seventeen Years Old'}, inplace=True)

In [45]:
df3_2 = df3.groupby(['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group']).sum()[['Chidlren Waiting on Adoption 31 August', 'Total Months Since Termination']]

In [46]:
df3_2.head()

Chidlren Waiting on Adoption 31 August  Total Months Since Termination
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                                                                              
2010        1-Lubbock Female African American Birth to Five Years Old                                              19                             191
                                              Six to Twelve Years Old                                              13                             496
                                              Thirteen to Seventeen Years Old                                       7                             345
                             Anglo            Birth to Five Years Old                                              25                             162
                                              Six to Twelve Years Old                                              20                             454

In [51]:
df3_2['Average Months Since Termination of Parental Rights_2'] = df3_2['Total Months Since Termination']/df3_2['Chidlren Waiting on Adoption 31 August']

In [52]:
df3_2.head()

Chidlren Waiting on Adoption 31 August  ...  Average Months Since Termination of Parental Rights_2
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                                                ...                                                       
2010        1-Lubbock Female African American Birth to Five Years Old                                              19  ...                                          10.052632    
                                              Six to Twelve Years Old                                              13  ...                                          38.153846    
                                              Thirteen to Seventeen Years Old                                       7  ...                                          49.285714    
                             Anglo            Birth to Five Years Old                                              25  ...                                           6.480000    
                                              Six to Twelve Years Old                                              20  ...                                          22.700000    

[5 rows x 3 columns]

In [53]:
final_df6 = pd.merge(df3_2, final_df5, on = ['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group'])

In [54]:
## WHY ISN'T THIS WORKING

final_df6.head()

Chidlren Waiting on Adoption 31 August  ...  Relative
Fiscal Year Region    Gender Race/Ethnicity   Age Group                                                                ...          
2010        1-Lubbock Female African American Birth to Five Years Old                                              19  ...      14.0
                                              Six to Twelve Years Old                                              13  ...       6.0
                                              Thirteen to Seventeen Years Old                                       7  ...       1.0
                             Anglo            Birth to Five Years Old                                              25  ...      18.0
                                              Six to Twelve Years Old                                              20  ...      18.0

[5 rows x 9 columns]

In [ ]:
df4.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Children in Adoption Placements
0,2019,1-Lubbock,Female,African American,Birth to Five Years Old,13
1,2019,1-Lubbock,Female,African American,Six to Twelve Years Old,5
2,2019,1-Lubbock,Female,African American,Thirteen to Seventeen Years Old,4
3,2019,1-Lubbock,Female,Anglo,Birth to Five Years Old,25
4,2019,1-Lubbock,Female,Anglo,Six to Twelve Years Old,15


In [56]:
final_df7 = pd.merge(df4, final_df6, on = ['Fiscal Year','Region','Gender','Race/Ethnicity','Age Group'])

In [57]:
final_df7.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Children in Adoption Placements,Chidlren Waiting on Adoption 31 August,Total Months Since Termination,Average Months Since Termination of Parental Rights_2,Removals,Victims,Foster Care,Other Substitute Care,Non-Relative,Relative
0,2019,1-Lubbock,Female,African American,Birth to Five Years Old,13,13,106,8.153846,10.0,10.0,14.0,10.0,15.0,9.0
1,2019,1-Lubbock,Female,African American,Six to Twelve Years Old,5,6,261,43.500000,14.0,14.0,20.0,5.0,20.0,5.0
2,2019,1-Lubbock,Female,African American,Thirteen to Seventeen Years Old,4,12,468,39.000000,8.0,10.0,15.0,9.0,18.0,6.0
3,2019,1-Lubbock,Female,Anglo,Birth to Five Years Old,25,17,69,4.058824,10.0,10.0,36.0,26.0,35.0,27.0
4,2019,1-Lubbock,Female,Anglo,Six to Twelve Years Old,15,24,244,10.166667,14.0,14.0,40.0,25.0,40.0,25.0


In [66]:
df5.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Disabling Condition,Children in Adoption Placements
0,2019,1-Lubbock,Female,African American,Birth to Five Years Old,Drug or Alcohol,6
1,2019,1-Lubbock,Female,Anglo,Birth to Five Years Old,Drug or Alcohol,4
2,2019,1-Lubbock,Female,Anglo,Six to Twelve Years Old,Drug or Alcohol,1
3,2019,1-Lubbock,Female,Hispanic,Birth to Five Years Old,Drug or Alcohol,6
4,2019,1-Lubbock,Male,Anglo,Birth to Five Years Old,Drug or Alcohol,9


In [72]:
df5.loc[(df5['Fiscal Year']=='2019') & (df5['Region']=='1-Lubbock') & (df5['Gender'] == 'Female') & (df5['Race/Ethnicity'] == 'African American') & (df5['Age Group'] == 'Birth to Five Years Old')]

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Fiscal Year,Region,Gender,Race/Ethnicity,Age Group,Disabling Condition,Children in Adoption Placements


In [59]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5896 entries, 0 to 5895
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Fiscal Year                      5896 non-null   int64 
 1   Region                           5896 non-null   object
 2   Gender                           5896 non-null   object
 3   Race/Ethnicity                   5896 non-null   object
 4   Age Group                        5848 non-null   object
 5   Disabling Condition              5896 non-null   object
 6   Children in Adoption Placements  5896 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 322.6+ KB


In [64]:
df5_2 = df5.groupby(['Fiscal Year', 'Region', 'Gender', 'Race/Ethnicity', 'Age Group','Disabling Condition']).count()['Children in Adoption Placements']

In [65]:
df5_2.head()

Fiscal Year  Region     Gender  Race/Ethnicity    Age Group                        Disabling Condition  
2010         1-Lubbock  Female  African American  Birth to Five Years Old          Drug or Alcohol          1
                                                                                   Learning Disability      1
                                                  Six to Twelve Years Old          Emotionally Disturbed    1
                                                                                   Learning Disability      1
                                                  Thirteen to Seventeen Years Old  Emotionally Disturbed    1
Name: Children in Adoption Placements, dtype: int64

In [ ]:
df6.head()

,Fiscal Year,Region,Gender,Race/Ethnicity,Same Race/Ethnicity as Adoptive Parent,Age Group,Consummated Adoptions
0,2019,1-Lubbock,Female,African American,Yes,Birth to Five Years Old,2
1,2019,1-Lubbock,Female,African American,Yes,Six to Twelve Years Old,1
2,2019,1-Lubbock,Female,African American,No,Birth to Five Years Old,10
3,2019,1-Lubbock,Female,African American,No,Six to Twelve Years Old,3
4,2019,1-Lubbock,Female,African American,No,Thirteen to Eighteen Years Old,3
